<a href="https://colab.research.google.com/github/Arya1790/RAG/blob/main/RAG_Application_Using_LangChain__Weviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

libraries:-

weaviate-client : vector database

LangChain : framework to build with LLMs

tiktoken : fast open-source tokenizer by OpenAI.
It's reversible and lossless, so you can convert tokens back into the original text

pypdf : free and open-source pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files

onnx : model 200% faster, which eliminates the need to use a GPU

In [1]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

In [2]:
!pip install -U langchain-community
!pip install sentence-transformers

In [3]:
# WEAVIATE cluster name rag
WEAVIATE_CLUSTER="https://ke7zibneqjq4wfbjhopcq.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="NGDzCoSwS6mUtFPfqvemlyQ4zYIhNWvPNnLV"

In [4]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [5]:
# fixing unicode error in google colab, the code snippet forces Python to use UTF-8 encoding as the preferred encoding, ensuring consistent text handling across different environments
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [6]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You 

In [7]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf", extract_images=True)
pages = loader.load()

In [8]:
pages[0]

Document(metadata={'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez?,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research;‡University College London;?New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate\nknowledge is still limited, and hence on knowledge-intensive tasks, their perfor-\nmance lags behind task-speciﬁc architectures. Additionally, providing provenance\nfor their decisions and updating their world knowledge remain open researc

In [9]:
# Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

docs[0]

Document(metadata={'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf', 'page': 0}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez?,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research;‡University College London;?New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate\nknowledge is still limited, and hence on knowledge-intensive tasks, their perfor-\nmance lags behind task-speciﬁc architectures. Additionally, providing provenance\nfor their decisions and updating their world knowledge remain open researc

In [10]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

by_text=False: This parameter indicates whether the embeddings are to be indexed by text or not.
When by_text=False, it means that the embeddings are directly provided and used as is, rather than being generated from the text within Weaviate.

In [11]:
print(vector_db.similarity_search("what is rag?", k=3)[0].page_content)

cases for NQ, where an extractive model would score 0%.
4.2 Abstractive Question Answering
As shown in Table 2, RAG-Sequence outperforms BART on Open MS-MARCO NLG by 2.6 Bleu
points and 2.6 Rouge-L points. RAG approaches state-of-the-art model performance, which is
impressive given that (i) those models access gold passages with speciﬁc information required to
generate the reference answer, (ii) many questions are unanswerable without the gold passages, and
(iii) not all questions are answerable from Wikipedia alone. Table 3 shows some generated answers
from our models. Qualitatively, we ﬁnd that RAG models hallucinate less and generate factually
correct text more often than BART. Later, we also show that RAG generations are more diverse than
BART generations (see §4.5).
4.3 Jeopardy Question Generation
Table 2 shows that RAG-Token performs better than RAG-Sequence on Jeopardy question generation,


In [12]:
print(vector_db.similarity_search("what is rag?", k=3)[1].page_content)

minimize the negative marginal log-likelihood of each target,P
j logp(yj|xj)using stochastic
gradient descent with Adam [ 28]. Updating the document encoder BERT dduring training is costly as
it requires the document index to be periodically updated as REALM does during pre-training [ 20].
We do not ﬁnd this step necessary for strong performance, and keep the document encoder (and
index) ﬁxed, only ﬁne-tuning the query encoder BERT qand the BART generator.
2.5 Decoding
At test time, RAG-Sequence and RAG-Token require different ways to approximate arg maxyp(y|x).
RAG-Token The RAG-Token model can be seen as a standard, autoregressive seq2seq genera-
tor with transition probability: p0
✓(yi|x, y 1:i 1)=P
z2top-k(p(·|x))p⌘(zi|x)p✓(yi|x, z i,y1:i 1)To
decode, we can plug p0
✓(yi|x, y 1:i 1)into a standard beam decoder.
RAG-Sequence For RAG-Sequence, the likelihood p(y|x)does not break into a conventional per-


In [13]:
print(vector_db.similarity_search("what is rag?", k=3)[2].page_content)

Broader Impact
This work offers several positive societal beneﬁts over previous work: the fact that it is more
strongly grounded in real factual knowledge (in this case Wikipedia) makes it “hallucinate” less
with generations that are more factual, and offers more control and interpretability. RAG could be
employed in a wide variety of scenarios with direct beneﬁt to society, for example by endowing it
with a medical index and asking it open-domain questions on that topic, or by helping people be more
effective at their jobs.
With these advantages also come potential downsides: Wikipedia, or any potential external knowledge
source, will probably never be entirely factual and completely devoid of bias. Since RAG can be
employed as a language model, similar concerns as for GPT-2 [ 50] are valid here, although arguably
to a lesser extent, including that it might be used to generate abuse, faked or misleading content in


In [14]:
print(
    vector_db.similarity_search(
        "what is attention?", k=3)
    )

[Document(metadata={'page': 14, 'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf'}, page_content='biases in sentence-pair classiﬁcation with elastic weight consolidation. ArXiv , abs/2004.14366,\n2020. URL https://arxiv.org/abs/2004.14366 .\n[58] Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,\nŁ ukasz Kaiser, and Illia Polosukhin. Attention is all you need. In I. Guyon, U. V. Luxburg,\nS. Bengio, H. Wallach, R. Fergus, S. Vishwanathan, and R. Garnett, editors, Advances in Neural\nInformation Processing Systems 30 , pages 5998–6008. Curran Associates, Inc., 2017. URL\nhttp://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf .\n[59] Ashwin Vijayakumar, Michael Cogswell, Ramprasaath Selvaraju, Qing Sun, Stefan Lee, David\nCrandall, and Dhruv Batra. Diverse beam search for improved description of complex scenes.\nAAAI Conference on Artiﬁcial Intelligence , 2018. URL https://www.aaai.o

In [15]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [16]:
prompt=ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))])

In [17]:
from langchain import HuggingFaceHub
from google.colab import userdata

huggingfacehub_api_token=userdata.get('HF')

In [18]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


 temperature = 1 means that the model’s predictions are based on its learned probabilities without additional scaling

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

RunnablePassthrough : In a chain or pipeline of operations sometimes you need a component that just passes the data along without altering it. This class/function can serve that purpose.

StrOutputParser: output is consistently formatted as a string

In [20]:
output_parser=StrOutputParser()

In [21]:
retriever=vector_db.as_retriever()

In [22]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [23]:
print(rag_chain.invoke("what is rag system?"))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: what is rag system?
Context: [Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf'}, page_content='Table 2 shows our results on FEVER. For 3-way classiﬁcation, RAG scores are within 4.3% of\nstate-of-the-art models, which are complex pipeline systems with domain-speciﬁc architectures and\nsubstantial engineering, trained using intermediate retrieval supervision, which RAG does not require.\n6'), Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf'}, page_content='cases for NQ, where an extractive model would score 0%.\n4.2 Abstractive Question Answering\nAs shown in 

In [24]:
print(rag_chain.invoke("How does the RAG model differ from traditional language generation models?"))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: How does the RAG model differ from traditional language generation models?
Context: [Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf'}, page_content='cases for NQ, where an extractive model would score 0%.\n4.2 Abstractive Question Answering\nAs shown in Table 2, RAG-Sequence outperforms BART on Open MS-MARCO NLG by 2.6 Bleu\npoints and 2.6 Rouge-L points. RAG approaches state-of-the-art model performance, which is\nimpressive given that (i) those models access gold passages with speciﬁc information required to\ngenerate the reference answer, (ii) many questions are unanswerable without the gold passages, and\n(iii) not all questions are answerable

In [34]:
output = rag_chain.invoke("How does the RAG model differ from traditional language generation models?")
start  = output.find('Answer:')+len('Answer:')
output[start:].strip()

'The RAG model differs from traditional language generation models in that it uses a retrieval-augmented approach to generate text. In this approach, the model retrieves relevant documents from a knowledge base and uses them to generate text. This allows the model to generate more factually correct text and to be more specific in its responses. The RAG model outperforms traditional language generation models on several benchmarks, including the MSMARCO NLG task and Jeopardy question generation'

In [26]:
# pdf, ppt, txt, xml, json, csv